In [1]:
#%matplotlib notebook
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from pyiomica_master import pyiomica as pio
from pyiomica_master.pyiomica import visualizationFunctions
from pyiomica_master.pyiomica import visibilityGraphCommunityDetection
import matplotlib as mpl
import networkx as nx
from tqdm import tqdm
from sklearn.model_selection import train_test_split

%matplotlib inline

Loading PyIOmica 1.3.3 (https://github.com/gmiaslab/pyiomica by G. Mias Lab)


In [2]:
data_path="/home/nvekariy/signal_code/datasets/Bonn_seizure_dataset/"

In [3]:
df_path=glob.glob(data_path+"***")

In [4]:
df_path

['/home/nvekariy/signal_code/datasets/Bonn_seizure_dataset/Set_A',
 '/home/nvekariy/signal_code/datasets/Bonn_seizure_dataset/Set_B',
 '/home/nvekariy/signal_code/datasets/Bonn_seizure_dataset/Set_C',
 '/home/nvekariy/signal_code/datasets/Bonn_seizure_dataset/Set_D',
 '/home/nvekariy/signal_code/datasets/Bonn_seizure_dataset/Set_E']

In [5]:
LABEL_C1 = 0
LABEL_C2 = 1

In [6]:
def load_data():
    data = []
    nbFiles = 0
    df=glob.glob(df_path[0]+"/***")
    for fname in tqdm(df):
        img = np.loadtxt(fname)
        data.append([img, np.array(LABEL_C1)])
        nbFiles+=1
        
    df1=glob.glob(df_path[-1]+"/***")
    for fname in tqdm(df1):
        img = np.loadtxt(fname)
        data.append([np.array(img), np.array(LABEL_C2)])
        nbFiles+=1
    
    return data

In [7]:
data = load_data()
print(len(data), "Files")

100%|██████████| 100/100 [00:02<00:00, 49.38it/s]

200 Files


In [8]:
from sklearn.utils import shuffle
data = shuffle(data,random_state=40)


X = np.array([d[0] for d in data])
Y = np.array([d[1] for d in data])



In [9]:
X.shape

(200, 4097)

In [10]:
def min_max_normalize(matrix):
    """
    Perform min-max normalization on a 2D array (matrix).
    """
    min_vals = np.min(matrix, axis=0)
    max_vals = np.max(matrix, axis=0)
    normalized = (matrix - min_vals) / (max_vals - min_vals)
    return normalized

In [11]:
X_n=np.array([min_max_normalize(i) for i in X])


In [12]:
X_temp=[]
Y_temp=[]
for i in range(len(X)):
        raw=X_n[i]
        X_temp.append(raw[:1024])
        X_temp.append(raw[1024:2048])
        X_temp.append(raw[2048:3072])
        X_temp.append(raw[3072:4096])
        
        Y_temp.extend(np.repeat(Y[i],4))

        

In [13]:
X_temp=np.array(X_temp)
Y_temp=np.array(Y_temp)

In [14]:
print(X_temp.shape)
print(Y_temp.shape)

(800, 1024)
(800,)


In [15]:
from sklearn.utils import shuffle
X_s,Y_s = shuffle(X_temp,Y_temp,random_state=40)


In [16]:
unique_elements, element_counts = np.unique(Y_s, return_counts=True)

print("Number of unique elements:", len(unique_elements))
print("Unique elements:", unique_elements)
print("Element counts:", element_counts)


Number of unique elements: 2
Unique elements: [0 1]
Element counts: [400 400]


In [17]:
def calculate_visibility(data):
    temp1=[]
    for p, i in enumerate(data):
        #print(f'Converting ==> {p+1} \r', end="",flush=True)
        vs_out = visibilityGraphCommunityDetection.createVisibilityGraph(i, range(len(i)), "dual_natural", weight='tan')
        vs_out=np.array(vs_out)
        temp1.append(vs_out)
        
    return temp1

In [18]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)


Number of CPU cores: 48


In [19]:
df=np.array_split(X_s, num_cores)


In [20]:
print(len(df[0]))

17


In [21]:
with Pool(processes=num_cores) as pool:
    results = pool.map(calculate_visibility, df)

pool.close()
pool.join()

In [22]:
x_adj=[]
for submatrix in results:
    x_adj.extend(submatrix)

In [23]:
print(len(x_adj))

800


In [24]:
x_adj=np.array(x_adj)

In [25]:
def degree(data): #The node strengths (or weighted degrees) 
    return np.average(np.sum(data, axis=0))

In [26]:
def runall(data):
    temp1=[]
    for i in data:
        G = nx.from_numpy_array(i, parallel_edges=False)
        
        cpl=[nx.average_shortest_path_length(G)]
        cc=list(nx.clustering(G).values())
        temp1.append(cpl+cc) #+cpl+ge+le+ac+cc
    return temp1

In [27]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)


df1=np.array_split(x_adj[:], num_cores)


print(len(df[0]))




Number of CPU cores: 48
17


In [28]:

import time
start_time = time.time()
with Pool(processes=num_cores) as pool1:
    results1 = pool1.map(runall, df1)

pool1.close()
pool1.join()

print("--- %s seconds ---" % (time.time() - start_time))


--- 44.83670163154602 seconds ---


In [29]:
x_g=[]
for submatrix in results1:
    x_g.extend(submatrix)

In [30]:
x_g=np.array(x_g)


In [31]:
print(x_g.shape)

(800, 1025)


In [32]:
x_train, x_test, y_train, y_test = train_test_split(x_g, Y_s, test_size=0.30, random_state=42)


In [33]:
print(x_train.shape)
print(x_test.shape)

(560, 1025)
(240, 1025)


In [34]:
import tensorflow as tf


2023-08-12 07:43:57.092749: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-12 07:43:59.906447: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-12 07:44:01.911500: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-

In [35]:
input_shape = (x_train.shape[1],1)
num_classes = 4
model3 = tf.keras.models.Sequential()
model3.add(tf.keras.layers.Conv1D(filters= 32, kernel_size=3, activation='relu',padding='same',input_shape= input_shape))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Flatten())

model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dropout(0.1))
model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

2023-08-12 07:44:19.798878: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-08-12 07:44:19.798941: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-12 07:44:19.798970: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c0188): /proc/driver/nvidia/version does not exist
2023-08-12 07:44:19.799990: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow

In [36]:
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1025, 32)          128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 512, 32)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 512, 32)           3104      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 256, 32)          0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 256, 32)           3104      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 128, 32)          0

In [37]:
model3.compile(loss='SparseCategoricalCrossentropy', optimizer='Adam', metrics=['acc'])


In [38]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model



checkpoint = ModelCheckpoint('F4.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


In [39]:
history_log = model3.fit(x_train, y_train, epochs=30, verbose=1, validation_data=(x_test, y_test), callbacks=[checkpoint])


Epoch 1/30
18/18 [==============================] - 2s 27ms/step - loss: 0.9174 - acc: 0.4982 - val_loss: 0.6934 - val_acc: 0.5375
Epoch 2/30
18/18 [==============================] - 0s 11ms/step - loss: 0.7330 - acc: 0.4661 - val_loss: 0.7047 - val_acc: 0.4625
Epoch 3/30
18/18 [==============================] - 0s 11ms/step - loss: 0.7027 - acc: 0.5375 - val_loss: 0.7127 - val_acc: 0.4625
Epoch 4/30
18/18 [==============================] - 0s 11ms/step - loss: 0.7015 - acc: 0.5179 - val_loss: 0.7001 - val_acc: 0.5375
Epoch 5/30
18/18 [==============================] - 0s 12ms/step - loss: 0.7167 - acc: 0.4839 - val_loss: 0.7060 - val_acc: 0.4625
Epoch 6/30
18/18 [==============================] - 0s 11ms/step - loss: 0.7051 - acc: 0.4804 - val_loss: 0.6959 - val_acc: 0.4625
Epoch 7/30
18/18 [==============================] - 0s 11ms/step - loss: 0.6899 - acc: 0.5214 - val_loss: 0.6846 - val_acc: 0.5375
Epoch 8/30
18/18 [==============================] - 0s 10ms/step - loss: 0.6787 - a

18/18 [==============================] - 0s 11ms/step - loss: 0.0602 - acc: 0.9839 - val_loss: 0.1641 - val_acc: 0.9542
Epoch 30/30
18/18 [==============================] - 0s 10ms/step - loss: 0.0670 - acc: 0.9696 - val_loss: 0.1265 - val_acc: 0.9750


In [40]:
model3.save_weights('/home/nvekariy/signal_code/New_bone_eeg_code_A_E_h/F4.h5')


In [41]:
model3.load_weights('/home/nvekariy/signal_code/New_bone_eeg_code_A_E_h/F4.h5')


In [42]:
predictions = model3.predict(x_test)
pred_arg = np.argmax(predictions, axis=1)

8/8 [==============================] - 0s 4ms/step


In [43]:
from sklearn.metrics import confusion_matrix
cnf_matrix=confusion_matrix(y_test,pred_arg)
print("cnf_matrix_2.0_70")
print("")
print(cnf_matrix)

cnf_matrix_2.0_70

[[110   1]
 [  5 124]]


In [44]:
from sklearn.metrics import accuracy_score, recall_score,precision_score,roc_auc_score,roc_curve,f1_score


In [45]:
print(accuracy_score(y_test,pred_arg)*100)
print(f1_score(y_test,pred_arg,average='macro')*100)
print(recall_score(y_test,pred_arg, average="weighted")*100)
print(precision_score(y_test,pred_arg, average="weighted")*100)

97.5
97.49146400947669
97.5
97.55913043478262
